## Wprowadzenie

Ta lekcja obejmie:
- Czym jest wywoływanie funkcji i do czego służy
- Jak utworzyć wywołanie funkcji za pomocą OpenAI
- Jak zintegrować wywołanie funkcji z aplikacją

## Cele nauki

Po ukończeniu tej lekcji będziesz wiedzieć jak i rozumieć:

- Cel używania wywoływania funkcji
- Konfigurację wywołania funkcji za pomocą usługi OpenAI
- Projektowanie skutecznych wywołań funkcji dla zastosowań Twojej aplikacji


## Zrozumienie wywołań funkcji

Na tę lekcję chcemy zbudować funkcję dla naszego startupu edukacyjnego, która pozwoli użytkownikom korzystać z chatbota do wyszukiwania kursów technicznych. Będziemy polecać kursy dopasowane do ich poziomu umiejętności, obecnej roli oraz interesującej technologii.

Do realizacji tego celu użyjemy kombinacji:
 - `OpenAI` do stworzenia doświadczenia czatu dla użytkownika
 - `Microsoft Learn Catalog API` do pomocy użytkownikom w znajdowaniu kursów na podstawie ich zapytań
 - `Function Calling` do przechwycenia zapytania użytkownika i przesłania go do funkcji wykonującej zapytanie do API.

Aby zacząć, przyjrzyjmy się, dlaczego w ogóle chcielibyśmy używać wywołań funkcji:

print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # uzyskaj nową odpowiedź od GPT, która może zobaczyć odpowiedź funkcji


print(second_response.choices[0].message)


### Dlaczego wywoływanie funkcji

Jeśli ukończyłeś już jakąkolwiek inną lekcję w tym kursie, prawdopodobnie rozumiesz moc korzystania z dużych modeli językowych (LLM). Mamy nadzieję, że dostrzegasz również niektóre z ich ograniczeń.

Wywoływanie funkcji to funkcja usługi OpenAI zaprojektowana, aby rozwiązać następujące wyzwania:

Niespójne formatowanie odpowiedzi:
- Przed wprowadzeniem wywoływania funkcji odpowiedzi z dużego modelu językowego były nieustrukturyzowane i niespójne. Programiści musieli pisać skomplikowany kod walidacyjny, aby obsłużyć każdą wariację w wyjściu.

Ograniczona integracja z danymi zewnętrznymi:
- Przed tą funkcją trudno było włączyć dane z innych części aplikacji do kontekstu czatu.

Poprzez standaryzację formatów odpowiedzi i umożliwienie płynnej integracji z danymi zewnętrznymi, wywoływanie funkcji upraszcza rozwój i zmniejsza potrzebę dodatkowej logiki walidacyjnej.

Użytkownicy nie mogli uzyskać odpowiedzi na pytania takie jak „Jaka jest aktualna pogoda w Sztokholmie?”. Dzieje się tak, ponieważ modele były ograniczone do czasu, na którym dane były trenowane.

Spójrzmy na poniższy przykład ilustrujący ten problem:

Załóżmy, że chcemy stworzyć bazę danych danych studentów, aby móc zasugerować im odpowiedni kurs. Poniżej mamy dwa opisy studentów, które są bardzo podobne pod względem zawartych danych.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finishing his studies."

Chcemy wysłać to do LLM, aby przetworzyć dane. Może to być później użyte w naszej aplikacji do wysłania tego do API lub zapisania w bazie danych.

Stwórzmy dwa identyczne polecenia, w których poinstruujemy LLM, jakich informacji jesteśmy zainteresowani:


Chcemy wysłać to do LLM, aby przeanalizował części ważne dla naszego produktu. Możemy więc stworzyć dwa identyczne polecenia, aby instruować LLM:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


Po utworzeniu tych dwóch promptów, wyślemy je do LLM za pomocą `openai.ChatCompletion`. Przechowujemy prompt w zmiennej `messages` i przypisujemy rolę `user`. Ma to na celu naśladowanie wiadomości od użytkownika wysyłanej do chatbota.


In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

: 

Teraz możemy wysłać oba zapytania do LLM i przeanalizować otrzymaną odpowiedź.


In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

Mimo że podpowiedzi są takie same, a opisy podobne, możemy otrzymać różne formaty właściwości `Grades`.

Jeśli uruchomisz powyższą komórkę wielokrotnie, format może być `3.7` lub `3.7 GPA`.

Wynika to z faktu, że LLM przyjmuje dane niestrukturalne w formie napisanego promptu i zwraca również dane niestrukturalne. Potrzebujemy mieć format strukturalny, aby wiedzieć, czego się spodziewać podczas przechowywania lub używania tych danych.

Korzystając z wywołań funkcji, możemy mieć pewność, że otrzymamy dane w formacie strukturalnym. Podczas używania wywołań funkcji LLM faktycznie nie wywołuje ani nie uruchamia żadnych funkcji. Zamiast tego tworzymy strukturę, której LLM ma się trzymać w swoich odpowiedziach. Następnie używamy tych strukturalnych odpowiedzi, aby wiedzieć, jaką funkcję uruchomić w naszych aplikacjach.


![Diagram przepływu wywołania funkcji](../../../../translated_images/pl/Function-Flow.083875364af4f4bb.png)


Następnie możemy wziąć to, co zwraca funkcja, i odesłać to z powrotem do LLM. LLM odpowie wtedy używając języka naturalnego, aby odpowiedzieć na zapytanie użytkownika.


### Przypadki użycia wywołań funkcji

**Wywoływanie narzędzi zewnętrznych**  
Chatboty świetnie nadają się do udzielania odpowiedzi na pytania użytkowników. Dzięki wywoływaniu funkcji chatboty mogą wykorzystywać wiadomości od użytkowników do wykonywania określonych zadań. Na przykład student może poprosić chatbota o „Wysłanie e-maila do mojego wykładowcy z informacją, że potrzebuję więcej pomocy z tym przedmiotem”. Może to wywołać funkcję `send_email(to: string, body: string)`

**Tworzenie zapytań do API lub bazy danych**  
Użytkownicy mogą wyszukiwać informacje za pomocą języka naturalnego, który jest konwertowany na sformatowane zapytanie lub żądanie API. Przykładem może być nauczyciel, który pyta „Którzy uczniowie ukończyli ostatnie zadanie”, co może wywołać funkcję o nazwie `get_completed(student_name: string, assignment: int, current_status: string)`

**Tworzenie danych strukturalnych**  
Użytkownicy mogą wziąć blok tekstu lub plik CSV i użyć LLM do wyodrębnienia z niego ważnych informacji. Na przykład student może przekształcić artykuł z Wikipedii o umowach pokojowych, aby stworzyć karty do nauki AI. Można to zrobić za pomocą funkcji `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`


## 2. Tworzenie pierwszego wywołania funkcji

Proces tworzenia wywołania funkcji obejmuje 3 główne kroki:
1. Wywołanie API Chat Completions z listą Twoich funkcji i wiadomością od użytkownika
2. Odczytanie odpowiedzi modelu w celu wykonania akcji, np. wywołania funkcji lub API
3. Ponowne wywołanie API Chat Completions z odpowiedzią z Twojej funkcji, aby wykorzystać te informacje do stworzenia odpowiedzi dla użytkownika.


![Przepływ wywołania funkcji](../../../../translated_images/pl/LLM-Flow.3285ed8caf4796d7.png)


### Elementy wywołania funkcji 

#### Dane wejściowe użytkownika 

Pierwszym krokiem jest utworzenie wiadomości użytkownika. Może to być dynamicznie przypisana wartość pobrana z pola tekstowego lub możesz przypisać wartość tutaj. Jeśli to Twój pierwszy raz z API Chat Completions, musimy zdefiniować `role` oraz `content` wiadomości. 

`role` może być `system` (tworzenie reguł), `assistant` (model) lub `user` (użytkownik końcowy). Dla wywołania funkcji przypiszemy to jako `user` oraz przykładowe pytanie. 


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### Tworzenie funkcji.

Następnie zdefiniujemy funkcję oraz parametry tej funkcji. Użyjemy tutaj tylko jednej funkcji o nazwie `search_courses`, ale możesz stworzyć wiele funkcji.

**Ważne**: Funkcje są dołączane do komunikatu systemowego do LLM i będą wliczane do dostępnej liczby tokenów, które masz do dyspozycji.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**Definicje** 

Struktura definicji funkcji ma wiele poziomów, z których każdy ma własne właściwości. Oto podział zagnieżdżonej struktury:

**Właściwości funkcji na najwyższym poziomie:**

`name` - Nazwa funkcji, którą chcemy wywołać. 

`description` - Opis działania funkcji. Tutaj ważne jest, aby być precyzyjnym i jasnym. 

`parameters` - Lista wartości i formatu, które model ma wygenerować w swojej odpowiedzi. 

**Właściwości obiektu parametrów:**

`type` - Typ danych obiektu parametrów (zazwyczaj "object")

`properties` - Lista konkretnych wartości, które model wykorzysta w swojej odpowiedzi. 

**Właściwości pojedynczych parametrów:**

`name` - Implicitnie zdefiniowane przez klucz właściwości (np. "role", "product", "level")

`type` - Typ danych tego konkretnego parametru (np. "string", "number", "boolean") 

`description` - Opis konkretnego parametru. 

**Właściwości opcjonalne:**

`required` - Tablica wymieniająca, które parametry są wymagane do wykonania wywołania funkcji.


### Wywołanie funkcji  
Po zdefiniowaniu funkcji, musimy teraz uwzględnić ją w wywołaniu API Chat Completion. Robimy to, dodając `functions` do żądania. W tym przypadku `functions=functions`.  

Istnieje również opcja ustawienia `function_call` na `auto`. Oznacza to, że pozwolimy LLM zdecydować, która funkcja powinna zostać wywołana na podstawie wiadomości użytkownika, zamiast przypisywać ją samodzielnie.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

Przyjrzyjmy się teraz odpowiedzi i zobaczmy, jak jest sformatowana:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

Widać, że nazwa funkcji została wywołana, a na podstawie wiadomości użytkownika LLM był w stanie znaleźć dane pasujące do argumentów funkcji.


## 3.Integrowanie wywołań funkcji z aplikacją. 


Po przetestowaniu sformatowanej odpowiedzi z LLM, możemy teraz zintegrować ją z aplikacją. 

### Zarządzanie przepływem 

Aby zintegrować to z naszą aplikacją, wykonajmy następujące kroki: 

Najpierw wykonajmy wywołanie do usług OpenAI i zapiszmy wiadomość w zmiennej o nazwie `response_message`. 


In [ ]:
response_message = response.choices[0].message

Teraz zdefiniujemy funkcję, która wywoła API Microsoft Learn, aby uzyskać listę kursów:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



Jako najlepszą praktykę, sprawdzimy następnie, czy model chce wywołać funkcję. Następnie utworzymy jedną z dostępnych funkcji i dopasujemy ją do wywoływanej funkcji.  
Następnie weźmiemy argumenty funkcji i odwzorujemy je na argumenty z LLM.

Na koniec dołączymy wiadomość wywołania funkcji oraz wartości zwrócone przez wiadomość `search_courses`. Dzięki temu LLM otrzyma wszystkie informacje potrzebne do  
odpowiedzi użytkownikowi w języku naturalnym.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



Teraz wyślemy zaktualizowaną wiadomość do LLM, abyśmy mogli otrzymać odpowiedź w języku naturalnym zamiast odpowiedzi w formacie JSON API.


In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Wyzwanie kodowania

Świetna robota! Aby kontynuować naukę wywoływania funkcji OpenAI, możesz zbudować: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst  
 - Więcej parametrów funkcji, które mogą pomóc uczącym się znaleźć więcej kursów. Dostępne parametry API znajdziesz tutaj:  
 - Utwórz kolejne wywołanie funkcji, które pobierze więcej informacji od uczącego się, na przykład jego język ojczysty  
 - Utwórz obsługę błędów, gdy wywołanie funkcji i/lub wywołanie API nie zwróci żadnych odpowiednich kursów


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Zastrzeżenie**:  
Niniejszy dokument został przetłumaczony za pomocą usługi tłumaczenia AI [Co-op Translator](https://github.com/Azure/co-op-translator). Mimo że dokładamy starań, aby tłumaczenie było jak najbardziej precyzyjne, prosimy mieć na uwadze, że automatyczne tłumaczenia mogą zawierać błędy lub nieścisłości. Oryginalny dokument w języku źródłowym powinien być uznawany za źródło autorytatywne. W przypadku informacji krytycznych zalecane jest skorzystanie z profesjonalnego tłumaczenia wykonanego przez człowieka. Nie ponosimy odpowiedzialności za jakiekolwiek nieporozumienia lub błędne interpretacje wynikające z korzystania z tego tłumaczenia.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
